In [9]:
# Import Application code to this notebook
%run ./Application.ipynb

Restaurant Data exported successfully to restaurants.csv
Events data exported successfully to restaurant_events.csv
Rating statistics exported successfully


C:\Users\junke\AppData\Local\Temp\ipykernel_18032\3893491828.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['restaurant.user_rating.aggregate_rating'] = filtered_df['restaurant.user_rating.aggregate_rating'].astype("float64")


In [10]:
import unittest
from unittest.mock import patch, MagicMock
import pandas as pd

In [4]:
class TestRestaurantDataProcessor(unittest.TestCase):
    def setUp(self):
        """Prepare resources for tests."""

        self.url = "https://raw.githubusercontent.com/Papagoat/brain-assessment/main/restaurant_data.json"
        self.country_code_file = "country_codes.csv"
        self.output_file_path = "processed_restaurants.csv"
        self.processor = RestaurantDataProcessor(self.url, self.country_code_file, self.output_file_path)

    def test_fetch_json_data(self):
        """Test fetching JSON data."""
        with patch('requests.get') as mock_get:
            mock_response = MagicMock()
            mock_response.ok = True
            mock_response.json.return_value = {"restaurants": []}
            mock_get.return_value = mock_response

            result = self.processor.fetch_json_data()
            self.assertIsNotNone(result)
            mock_get.assert_called_once_with(self.url)

            mock_response.ok = False
            result = self.processor.fetch_json_data()
            self.assertIsNone(result)

    def test_read_country_code_data(self):
        """Test reading country code data."""
        with patch('pandas.read_csv') as mock_read_csv:
            mock_read_csv.return_value = pd.DataFrame(data={"Country Code": ["IN"], "Country": ["India"]})
            result = self.processor.read_country_code_data()
            self.assertIsNotNone(result)
            mock_read_csv.assert_called_once_with(self.country_code_file)

            mock_read_csv.side_effect = FileNotFoundError
            result = self.processor.read_country_code_data()
            self.assertIsNone(result)

    def test_merge_data(self):
        """Test merging JSON and CSV data."""
        parsed_data = {"restaurants": [{"restaurant": {"location": {"country_id": "IN"}}}]}
        country_code_df = pd.DataFrame(data={"Country Code": ["IN"], "Country": ["India"]})
        result = self.processor.merge_data(parsed_data, country_code_df)
        self.assertIn("Country", result.columns)

    def test_process_merged_data(self):
        """Test processing merged DataFrame."""
        merged_df = pd.DataFrame(data={
            "restaurant.R.res_id": [1],
            "restaurant.name": ["Test Restaurant"],
            "Country": ["India"],
            "restaurant.location.city": ["Test City"],
            "restaurant.user_rating.votes": [100],
            "restaurant.user_rating.aggregate_rating": ["4.5"],
            "restaurant.cuisines": ["Indian"]
        })
        result = self.processor.process_merged_data(merged_df)
        self.assertIn("Restaurant Name", result.columns)
        self.assertTrue(result["User Aggregate Rating"].dtype == float)

    def test_export_df_to_csv(self):
        """Test exporting DataFrame to CSV."""
        with patch('pandas.DataFrame.to_csv') as mock_to_csv:
            df = pd.DataFrame()
            self.processor.export_df_to_csv(df)
            mock_to_csv.assert_called_once_with(self.output_file_path, index=False)

    def test_run(self):
        """Test the run method orchestrating the workflow."""
        with patch.object(RestaurantDataProcessor, 'fetch_json_data') as mock_fetch, \
             patch.object(RestaurantDataProcessor, 'read_country_code_data') as mock_read, \
             patch.object(RestaurantDataProcessor, 'merge_data') as mock_merge, \
             patch.object(RestaurantDataProcessor, 'process_merged_data') as mock_process, \
             patch.object(RestaurantDataProcessor, 'export_df_to_csv') as mock_export:
                
            mock_fetch.return_value = {"restaurants": []}
            mock_read.return_value = pd.DataFrame()
            mock_merge.return_value = pd.DataFrame()
            mock_process.return_value = pd.DataFrame()

            self.processor.run()

            mock_fetch.assert_called_once()
            mock_read.assert_called_once()
            mock_merge.assert_called_once()
            mock_process.assert_called_once()
            mock_export.assert_called_once()


In [8]:
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)